In [1]:
import sys
!{sys.executable} -m pip install wwo_hist

In [2]:
import numpy as np
import pandas as pd

import wwo_hist
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

In [5]:
df = pd.read_csv(
    "simulation_data_v2.csv"
    )
df.head()

,Hour,Point,Energy,Timestamp,Date,Day of Week,Out_of_Office,Presurvey,Weekly_Survey,Before_Game_Baseline,In_Game_Baseline
0,8,0.000000,58.383468,2018-09-20 08:00:00,2018-09-20,Thursday,0,4.82,1.79,8.531474,NaN
1,9,0.000000,88.785622,2018-09-20 09:00:00,2018-09-20,Thursday,0,4.82,1.79,11.331923,NaN
2,10,10.000000,76.836119,2018-09-20 10:00:00,2018-09-20,Thursday,0,4.82,1.79,26.189487,NaN
3,11,10.000000,90.501904,2018-09-20 11:00:00,2018-09-20,Thursday,0,4.82,1.79,29.648204,NaN
4,12,9.871989,53.371114,2018-09-20 12:00:00,2018-09-20,Thursday,0,4.82,1.79,29.592628,NaN


# Adding weather data

We first get the range of timestamps, so we know how much weather data to get

In [6]:
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df['Timestamp'].min(), df['Timestamp'].max()

(Timestamp('2018-09-20 08:00:00'), Timestamp('2023-05-03 14:00:00'))

We only want to grab a month's worth of data.

In [7]:
df = df[df['Timestamp'] < pd.Timestamp('2018-10-20 08:00:00')]
df['Timestamp'].min(), df['Timestamp'].max()

(Timestamp('2018-09-20 08:00:00'), Timestamp('2018-10-19 17:00:00'))

In [13]:
frequency = 1
start_date = '20-SEPT-2018'
end_date = '19-OCT-2018'
api_key = 'ac2aff5d06ab4bc2bdb181823201106'
location_list = ['singapore']

import os
file_exists = os.path.exists("singapore.csv")

hist_weather_data = (wwo_hist.retrieve_hist_data(api_key,
                                location_list,
                                start_date,
                                end_date,
                                frequency,
                                location_label = False,
                                export_csv = True,
                                store_df = True)[0] 
                     if not file_exists
                     else pd.read_csv("singapore.csv"))

In [14]:
weather_df.head()

,date_time,maxtempC,mintempC,totalSnow_cm,sunHour,uvIndex,moon_illumination,moonrise,moonset,sunrise,...,WindGustKmph,cloudcover,humidity,precipMM,pressure,tempC,visibility,winddirDegree,windspeedKmph,location
0,2018-09-20 00:00:00,33,25,0.0,8.9,7,72,03:35 PM,03:06 AM,06:55 AM,...,4,29,83,0.0,1010,25,10,26,3,singapore
1,2018-09-20 01:00:00,33,25,0.0,8.9,7,72,03:35 PM,03:06 AM,06:55 AM,...,3,27,84,0.0,1010,25,10,27,2,singapore
2,2018-09-20 02:00:00,33,25,0.0,8.9,7,72,03:35 PM,03:06 AM,06:55 AM,...,3,24,84,0.0,1009,25,10,28,2,singapore
3,2018-09-20 03:00:00,33,25,0.0,8.9,7,72,03:35 PM,03:06 AM,06:55 AM,...,2,22,84,0.0,1009,25,10,30,1,singapore
4,2018-09-20 04:00:00,33,25,0.0,8.9,7,72,03:35 PM,03:06 AM,06:55 AM,...,2,25,83,0.0,1010,25,10,130,1,singapore


### Taking temperature, precipitation, humidity, and joining on the main dataframe.

In [15]:
weather_subset_df = (weather_df[['date_time', 'tempC', 'precipMM', 'humidity']]
                    .rename({"date_time": "Timestamp"}, axis=1))
weather_subset_df.head()

,Timestamp,tempC,precipMM,humidity
0,2018-09-20 00:00:00,25,0.0,83
1,2018-09-20 01:00:00,25,0.0,84
2,2018-09-20 02:00:00,25,0.0,84
3,2018-09-20 03:00:00,25,0.0,84
4,2018-09-20 04:00:00,25,0.0,83


In [16]:
df = df.merge(right=weather_subset_df, on="Timestamp", how="left")
df.head()

,Hour,Point,Energy,Timestamp,Date,Day of Week,Out_of_Office,Presurvey,Weekly_Survey,Before_Game_Baseline,In_Game_Baseline,tempC,precipMM,humidity
0,8,0.000000,58.383468,2018-09-20 08:00:00,2018-09-20,Thursday,0,4.82,1.79,8.531474,NaN,31,0.1,61
1,9,0.000000,88.785622,2018-09-20 09:00:00,2018-09-20,Thursday,0,4.82,1.79,11.331923,NaN,33,0.1,52
2,10,10.000000,76.836119,2018-09-20 10:00:00,2018-09-20,Thursday,0,4.82,1.79,26.189487,NaN,33,0.1,54
3,11,10.000000,90.501904,2018-09-20 11:00:00,2018-09-20,Thursday,0,4.82,1.79,29.648204,NaN,32,0.0,56
4,12,9.871989,53.371114,2018-09-20 12:00:00,2018-09-20,Thursday,0,4.82,1.79,29.592628,NaN,32,0.0,58


# Preprocessing

## First Check Stationarity of Time Series

Time series are stationary if they do not have trend or seasonal effects. Summary statistics calculated on the time series are consistent over time, like the mean or the variance of the observations.

When a time series is stationary, it can be easier to model. Statistical modeling methods assume or require the time series to be stationary to be effective.

We use the Dickey-Fuller test to test of our time series is stationary.

### Dickey-Fuller Test

In [66]:
from statsmodels.tsa.stattools import adfuller

In [67]:
X = df['Energy']

In [68]:
result = adfuller(X)
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))

ADF Statistic: -5.133726
p-value: 0.000012
Critical Values:
	1%: -3.454
	5%: -2.872
	10%: -2.572


Since the p-value is less than 0.05, we can reject the null-hypothesis, and proceed with our analysis.

In [20]:
encoder = LabelEncoder()
df["Day of Week"] = encoder.fit_transform(df["Day of Week"])

## Remove extra columns

In [21]:
df = (df[['Timestamp', 'Point', 'Day of Week', 'Out_of_Office', 'tempC', 'precipMM', 'humidity', 'Energy']]
      .set_index('Timestamp'))
df.head()

,Point,Day of Week,Out_of_Office,tempC,precipMM,humidity,Energy
Timestamp,,,,,,,
2018-09-20 08:00:00,0.000000,4,0,31,0.1,61,58.383468
2018-09-20 09:00:00,0.000000,4,0,33,0.1,52,88.785622
2018-09-20 10:00:00,10.000000,4,0,33,0.1,54,76.836119
2018-09-20 11:00:00,10.000000,4,0,32,0.0,56,90.501904
2018-09-20 12:00:00,9.871989,4,0,32,0.0,58,53.371114


## Check for nan values

In [22]:
df.isnull().values.any()

False

## `series_to_supervised` (Time lag function)

In [32]:
def series_to_supervised(data,
                         n_in,
                         target_features,
                         col_names=None,
                         n_out=1,
                         dropnan=True,
                         initial_excluded_timesteps=0,
                         only_current_timestep_features=[]):
    """Takes time series data and converts it into a supervised learning
    problem framework.

        Parameters:
            - data (pd.Dataframe) -- the time series data to be converted.
            - n_in (int) -- Number of time steps to use as lag for the feature 
                matrix
            - col_names (List[str]) -- list of strings to use as column names,
                that get converted into features for each time lag
            - target_features (List[str]) -- List of features that will be used
                as dependent variables in the target matrix.
            - n_out (int) -- Number of time steps to use as lag for the target
                matrix
            - dropnan (bool) -- Whether to drop nan values
            - initial_excluded_timesteps (int) -- The number of input timesteps to 
                ignore before starting the time lag.
            - only_current_timestep_features (List[str]) -- Features that should
            only be included in the current timestep, not any before (e.g.) to avoid
            unintended dependencies
            
            - [Planned] exclude_current_day (bool) -- Whether to include values 
                from the current day. If this parameter is false, then the time lag 
                will always start with the day preceding the current time 
                step.
                
        Outputs:
            - (X, y): (Feature matrix, target matrix)
    """
    
    if col_names is None:
        col_names = data.columns
        
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()      
        

    # (t-n, ... t-1) --> i.e. steps into the past
    for i in range(n_in + initial_excluded_timesteps, initial_excluded_timesteps, -1):
        cols.append(df.shift(i))
        names += [('%s(t-%d)' % (col, i)) for col in col_names]

    # (t, t+1, ... t+n) --> i.e. steps into the future
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('%s(t)' % (col)) for col in col_names]
        else:
            names += [('%s(t + %d)' % (col, i)) for col in col_names]

    # concat
    agg = pd.concat(cols, axis=1)
    agg.columns = names

    # dropnan
    if dropnan:
        agg.dropna(inplace=True)
    return agg

## Separate X and Y variables

In [33]:
df_y = df["Energy"]
df_X = df.drop("Energy", axis = 1)

values_y = df_y.values
values_X = df_X.values
scaler_X = MinMaxScaler(feature_range = (0, 1))
scaler_y = MinMaxScaler(feature_range = (0, 1))

values_y = values_y.astype("float32")
values_X = values_X.astype("float32")

scaled_X = scaler_X.fit_transform(values_X)
scaled_y = scaler_y.fit_transform(values_y.reshape(-1, 1))

scaled = np.concatenate((scaled_X, scaled_y), axis = 1)

reframed = series_to_supervised(data = scaled, col_names = df.columns.tolist(), n_in = 18, n_out = 1)
reframed.head()

,Point(t-18),Day of Week(t-18),Out_of_Office(t-18),tempC(t-18),precipMM(t-18),humidity(t-18),Energy(t-18),Point(t-17),Day of Week(t-17),Out_of_Office(t-17),...,precipMM(t-1),humidity(t-1),Energy(t-1),Point(t),Day of Week(t),Out_of_Office(t),tempC(t),precipMM(t),humidity(t),Energy(t)
18,0.000000,0.666667,0.0,0.666667,0.022727,0.300,0.110800,0.000000,0.666667,0.0,...,0.795455,0.525,0.692190,0.000000e+00,0.000000,0.0,0.333333,0.386364,0.600,0.464206
19,0.000000,0.666667,0.0,0.888889,0.022727,0.075,0.168498,1.000000,0.666667,0.0,...,0.386364,0.600,0.464206,0.000000e+00,0.000000,0.0,0.222222,0.000000,0.675,0.251754
20,1.000000,0.666667,0.0,0.888889,0.022727,0.125,0.145820,1.000000,0.666667,0.0,...,0.000000,0.675,0.251754,1.907349e-07,0.333333,0.0,0.444444,0.000000,0.550,0.193240
21,1.000000,0.666667,0.0,0.777778,0.000000,0.175,0.171755,0.987199,0.666667,0.0,...,0.000000,0.550,0.193240,0.000000e+00,0.333333,0.0,0.555555,0.000000,0.400,0.324154
22,0.987199,0.666667,0.0,0.777778,0.000000,0.225,0.101288,0.000000,0.666667,0.0,...,0.000000,0.400,0.324154,1.000000e+00,0.333333,0.0,0.666667,0.000000,0.375,0.260801


# Benchmark Testing

## Model evaluation metrics

In [29]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt

def rmse(predicted_y, actual_y):
    return sqrt(mean_squared_error(actual_y, predicted_y))

def cv_rmse(predicted_y, actual_y):
    """This measures the coefficient of variation, which is the RMSE normalized by 
    the mean of the measured values and quantifies typical size of the error 
    relative to the mean of the observations. A high CV score indicates that a 
    model has a high error range.
    """
    return rmse(predicted_y, actual_y) / sqrt(np.mean(actual_y))

def mae(predicted_y, actual_y):
    return mean_absolute_error(actual_y, predicted_y)

evaluation_methods = [rmse, cv_rmse, mae]

## KNN

## Random Forests

## Gradient Boosting

## Standard NN

## Extra Trees Regressor

# Model Selection

## Genetic algorithm to find optimal amount of time lag

## Exploring AutoML

## Exploring Bayesian Optimization

# GA-augmented LSTM-RNN model

# LSTM-GAN Testing